In [1]:
import pandas as pd
import gzip
import json

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('sentiwordnet')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/triet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/triet/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /home/triet/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /home/triet/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/triet/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/triet/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [1]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(torch._C._cuda_getCompiledVersion())
cuda_id = torch.cuda.current_device()
cuda_name = torch.cuda.get_device_name(cuda_id)
print(cuda_id)
print(cuda_name)
print(device)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

12.1
False


/home/triet/Recommendation_System/DeepCGSR_triet/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [15]:
from sklearn.model_selection import train_test_split
from gensim.models import LdaModel
from gensim import corpora
from gensim.models import CoherenceModel
from nltk.tokenize import word_tokenize
import numpy as np
from nltk.corpus import stopwords

# Assume 'reviews' is your dataset containing reviews
# Tạo tập "reviews" nếu chưa có
reviews = []

# 10 đánh giá về phim
new_reviews = [
    "Great movie! Excellent plot and amazing performances by the cast.",
    "The film was disappointing. Weak storyline and poor acting.",
    "Absolutely loved it! A must-watch movie for everyone.",
    "The cinematography was outstanding, but the script lacked depth.",
    "A decent film, but the pacing was a bit slow for my liking.",
    "Incredible movie! Captivating from start to finish.",
    "Disappointed with the ending. It felt rushed and unresolved.",
    "The acting was superb, but the plot twists were predictable.",
    "An enjoyable film with some touching moments.",
    "This movie exceeded my expectations. Brilliant direction and storytelling."
]

# Thêm đánh giá vào tập "reviews"
reviews.extend(new_reviews)

# Chia tập dữ liệu thành tập huấn luyện và tập kiểm tra (80-20)
train_data, test_data = train_test_split(reviews, test_size=0.2, random_state=42)

# Tokenize reviews
tokenized_train_data = [word_tokenize(review.lower()) for review in train_data]
tokenized_test_data = [word_tokenize(review.lower()) for review in test_data]


# Danh sách stop words từ NLTK
stop_words = set(stopwords.words('english'))

# Tạo từ điển từ dữ liệu văn bản loại bỏ stop words
tokenized_train_data = [[token for token in doc if token not in stop_words] for doc in tokenized_train_data]

# Tạo từ điển từ dữ liệu văn bản
dictionary = corpora.Dictionary(tokenized_train_data)
for i in dictionary:
    print(dictionary[i])
# Chuyển đổi dữ liệu văn bản thành BoW
corpus_train = [dictionary.doc2bow(doc) for doc in tokenized_train_data]
corpus_test = [dictionary.doc2bow(doc) for doc in tokenized_test_data]

# Train mô hình LDA
num_topics = 10  # Số lượng chủ đề bạn muốn trích xuất
lda_model = LdaModel(corpus_train, num_topics=num_topics, id2word=dictionary, passes=15)

# Đánh giá mô hình trên tập kiểm tra
perplexity = lda_model.log_perplexity(corpus_test)
print("Perplexity on test data:", perplexity)

coherence_model = CoherenceModel(model=lda_model, texts=tokenized_test_data, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print("Coherence Score on test data:", coherence_score)


!
.
captivating
finish
incredible
movie
start
amazing
cast
excellent
great
performances
plot
,
acting
predictable
superb
twists
absolutely
everyone
loved
must-watch
brilliant
direction
exceeded
expectations
storytelling
bit
decent
film
liking
pacing
slow
cinematography
depth
lacked
outstanding
script
disappointed
ending
felt
rushed
unresolved
Perplexity on test data: -17.124323387940724


Coherence Score on test data: nan


In [7]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [17]:
df = getDF('data/All_Beauty_5.json.gz')

In [4]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"09 1, 2016",A3CIUOJXQ5VDQ2,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Shelly F,As advertised. Reasonably priced,Five Stars,1472688000,NaN,NaN
1,5.0,True,"11 14, 2013",A3H7T87S984REU,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",houserules18,Like the oder and the feel when I put it on my...,Good for the face,1384387200,NaN,NaN
2,1.0,True,"08 18, 2013",A3J034YH7UG4KT,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Adam,I bought this to smell nice after I shave. Wh...,Smells awful,1376784000,NaN,NaN
3,5.0,False,"05 3, 2011",A2UEO5XR3598GI,B0000530HU,"{'Size:': ' 7.0 oz', 'Flavor:': ' Classic Ice ...",Rich K,HEY!! I am an Aqua Velva Man and absolutely lo...,Truth is There IS Nothing Like an AQUA VELVA MAN.,1304380800,25,NaN
4,5.0,True,"05 6, 2011",A3SFRT223XXWF7,B00006L9LC,{'Size:': ' 200ml/6.7oz'},C. C. Christian,If you ever want to feel pampered by a shampoo...,Bvlgari Shampoo,1304640000,3,NaN


In [5]:
reviewers = sorted(df['reviewerID'].unique())

In [6]:
items = sorted(df['asin'].unique())

In [7]:
scores = sorted(df['overall'].unique())

In [8]:
scores

[1.0, 2.0, 3.0, 4.0, 5.0]

In [9]:
len(reviewers),len(items)

(991, 85)

In [37]:
rows = []
cols = []
data = []
for item in df.itertuples():
    r = item[1]
    u = item[4]
    i = item[5]
    iu = reviewers.index(u)
    ii = items.index(i)
    rows.append(iu)
    cols.append(ii)
    data.append(r)

In [38]:
len(rows)

5269

In [39]:
import numpy as np

In [40]:
ratings = np.zeros((len(reviewers),len(items)))

In [41]:
ratings.shape

(991, 85)

In [42]:
for r,c,d in zip(rows,cols,data):
    ratings[int(r),int(c)] = d

In [43]:
ratings[100]

array([0., 0., 0., 0., 0., 5., 5., 0., 0., 0., 0., 0., 0., 5., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [46]:
import scipy.sparse as sparse
def create_sparse_matrix(data, rows, cols, len_user,len_item):
    """ Returns a sparse utility matrix""" 
    return sparse.csc_matrix((data,(rows, cols)),shape=(len_user, len_item))

In [47]:
sparse_ratings = create_sparse_matrix(data,rows,cols,len(reviewers),len(items))

In [45]:
def create_embeddings(n, K):
    return 6*np.random.random((n, K)) / K

In [99]:
def cost(data,rows,cols, emb_user, emb_item):
    p_predict = predict(emb_user, emb_item)
    p_data = [p_predict[r][c] for r,c in zip(rows,cols)]
    predicted = create_sparse_matrix(p_data,rows,cols, emb_user.shape[0], emb_item.shape[0])
    return np.sum((sparse_ratings-predicted).power(2))/len(data)

In [84]:
def predict(emb_user, emb_item):
    p_ratings = np.dot(emb_user,emb_item.transpose())
    return p_ratings

In [101]:
beta = 0.9
lmbda = 0.0002
k = 10
learning_rate=0.01
iterations=2000
u_dim = len(reviewers)
i_dim = len(items)

In [49]:
import torch

In [103]:
emb_user = create_embeddings(u_dim,k)
emb_item = create_embeddings(i_dim,k)

In [104]:
emb_user.shape[0],emb_item.shape[0]

(991, 85)

In [97]:
def gradient(df,rows,cols, emb_user, emb_item):
    """ Computes the gradient for user and item embeddings"""
    Y = sparse_ratings
    p_predict = predict(emb_user, emb_item)
#     print(p_predict.shape)
    p_data = [p_predict[r][c] for r,c in zip(rows,cols)]
    predicted = create_sparse_matrix(p_data,rows,cols, emb_user.shape[0], emb_item.shape[0])
    delta =(Y-predicted)
    grad_user = (-2/df.shape[0])*(delta*emb_item) + 2*lmbda*emb_user
    grad_item = (-2/df.shape[0])*(delta.T*emb_user) + 2*lmbda*emb_item
    return grad_user, grad_item

In [105]:
grad_user, grad_item = gradient(df,rows,cols, emb_user, emb_item)
v_user = grad_user
v_item = grad_item
for i in range(iterations):
    grad_user, grad_item = gradient(df,rows,cols, emb_user, emb_item)
    v_user = beta*v_user + (1-beta)*grad_user
    v_item = beta*v_item + (1-beta)*grad_item
    emb_user = emb_user - learning_rate*v_user
    emb_item = emb_item - learning_rate*v_item
    if(not (i+1)%50):
        print("\niteration", i+1, ":")
        print("train mse:",  cost(data,rows,cols, emb_user, emb_item))


iteration 50 :
train mse: 16.98350604199123

iteration 100 :
train mse: 13.420272090425673

iteration 150 :
train mse: 11.213063711743391

iteration 200 :
train mse: 9.760890983310453

iteration 250 :
train mse: 8.740989938899006

iteration 300 :
train mse: 7.978743301094478

iteration 350 :
train mse: 7.378341677292116

iteration 400 :
train mse: 6.8858506858924455

iteration 450 :
train mse: 6.469685966716291

iteration 500 :
train mse: 6.110368414980986

iteration 550 :
train mse: 5.795160898468652

iteration 600 :
train mse: 5.5152424918544245

iteration 650 :
train mse: 5.264192866717178

iteration 700 :
train mse: 5.037153457614434

iteration 750 :
train mse: 4.830341690379502

iteration 800 :
train mse: 4.640753403457753

iteration 850 :
train mse: 4.46596914819623

iteration 900 :
train mse: 4.304020658226391

iteration 950 :
train mse: 4.153294247876669

iteration 1000 :
train mse: 4.012458300548648

iteration 1050 :
train mse: 3.8804073867422764

iteration 1100 :
train mse: 